In [3]:
import pandas as pd
from tqdm.auto import tqdm
from util import *
from stationUtil import *
from graph import *
from glob import glob

In [4]:
# DEMAND_GRAPH_PATH = "../trimmed_data/demand_graph.pickle"
# demand = loadPickle(DEMAND_GRAPH_PATH)
f_list = glob("../trimmed_data/A001_20*.csv.*")
demand = [loadPickle(p) for p in f_list]
demand[0].keys().__len__()
demand[0][(95,302)]

91

In [5]:
# 아무것도 제외 하지 않으면 완행열차와 동일하다.
UNITFACTOR_P1KM_TO_1KM = 1 / 10
UNITFACTOR_1SEC_TO_1HOUR = 1 / 3600
EXCLUDE_STATION = []
g_time = time_weight_graph(exclude_list=EXCLUDE_STATION)
g_dist = dist_weight_graph()

sCode_start, sCode_end = 95, 301
calc_total_path_weight(g_time, sCode_start, sCode_end), calc_total_path_weight(
    g_dist, sCode_start, sCode_end
) * sum([d[(95,302)] for d in demand]) / len(demand)  # (Second, 0.1km)

(4547.9284707102415, 41435.181818181816)

In [6]:
def od_metric(scode_start, scode_end):
    month_size = len(demand)
    d_list = []
    for d in demand:
        try:
            d_list.append(d[(scode_start, scode_end)])
        except:
            continue
    return sum(d_list)/month_size

def unit_tranfer_flux_metric(scode_start, scode_end, g_dist=g_dist):
    path_dist = (
        calc_total_path_weight(g_dist, scode_start, scode_end) * UNITFACTOR_P1KM_TO_1KM
    )
    path_od = od_metric(scode_start, scode_end)
    return (path_dist, path_od)

od_metric(95,302) == sum([d[(95,302)] for d in demand]) / len(demand)

True

In [13]:
# Calc Flux Metric
dist = {}
od = {}
flux_metric = {}
s_namecode = loadPickle("../trimmed_data/stationNameCode.pickel").values()
s_namecode = set(s_namecode)

for target_scode in tqdm(s_namecode):
    for calc_scode in s_namecode:
        if target_scode == calc_scode:
            continue
        p_dist, p_od = unit_tranfer_flux_metric(target_scode, calc_scode)
        flux_metric[(target_scode, calc_scode)] = p_dist * p_od
        dist[(target_scode, calc_scode)] = p_dist
        od[(target_scode, calc_scode)] = p_od

od_sum = {scode: [0, 0] for scode in s_namecode}
sum_flux_IO = {scode: [0, 0] for scode in s_namecode}
for target_scode in tqdm(s_namecode):
    population_IN, population_OUT = [], []
    sflux_IN, sflux_OUT = [], []
    for calc_scode in s_namecode:
        if target_scode == calc_scode:
            continue
        sflux_IN.append(flux_metric[(target_scode, calc_scode)])
        sflux_OUT.append(flux_metric[(calc_scode, target_scode)])
        population_IN.append(od[(target_scode, calc_scode)])
        population_OUT.append(od[(calc_scode, target_scode)])

    valid_IN = sum([int(s != 0) for s in sflux_IN])
    valid_OUT = sum([int(s != 0) for s in sflux_OUT])
    sum_flux_IO[target_scode][0] = (
        sum(sflux_IN) / valid_IN if not valid_IN == 0 else sum(sflux_IN) * valid_IN
    )
    sum_flux_IO[target_scode][1] = (
        sum(sflux_OUT) / valid_OUT if not valid_OUT == 0 else sum(sflux_IN) * valid_OUT
    )
    
    od_sum[target_scode][0] = sum(population_IN)/len(population_IN)
    od_sum[target_scode][1] = sum(population_OUT)/len(population_IN)

sum_flux_IO[301] = sum_flux_IO[208]
sum_flux_IO[401] = sum_flux_IO[309]
print(flux_metric.__len__())

import pickle
with open("../database/SUBWAY_OD_RAW.pkl","wb") as f:
    pickle.dump(flux_metric,f)

  0%|          | 0/114 [00:00<?, ?it/s]

100%|██████████| 114/114 [00:00<00:00, 5206.63it/s]

12882


In [8]:
SUM_FLUX_METRIC = pd.DataFrame(sum_flux_IO).T
SUM_FLUX_METRIC = SUM_FLUX_METRIC.rename(columns={0: "IN_FLUX", 1: "OUT_FLUX"})
SUM_FLUX_METRIC.index.name = "scode"

SUM_FLUX_METRIC.to_json("../database/SUM_FLUX_IO_METRIC.json")
SUM_FLUX_METRIC.loc[line_num_to_range(4), :].head()

,IN_FLUX,OUT_FLUX
scode,,
401,15395.282555,14665.584603
402,2153.854709,2188.623260
403,3232.187633,3020.024980
404,1718.660197,1529.723342
405,3011.642834,2661.703194


In [9]:
SAVE_PATH = "../database"
IMPLICIT_METRIC = pd.read_csv(f"{SAVE_PATH}/IMPLICIT_METRIC.csv", encoding="UTF-8")
IMPLICIT_METRIC = IMPLICIT_METRIC.set_index("scode")

IMPLICIT_METRIC.to_json("../database/IMPLICIT_METRIC.json")
IMPLICIT_METRIC.loc[line_num_to_range(4), :].head()

,직장인구,주거인구,유동인구
scode,,,
401,22686.245732,53436.484089,145891.733631
402,28131.387220,57905.960927,153513.978557
403,33821.496231,60586.596287,155414.195953
404,33136.589711,59439.265459,150782.722665
405,29224.610736,56025.691439,136535.043838
